# Building second level models using _nipype_ and _SPM12_

## Base functionality for _megameta_ project

-------
#### History

* 4/15/19 mbod - incorporate function to read the 2nd level JSON model config
* 4/9/19 mbod - modify template to work with fmriprep processed data
* 3/20/19 mbod - initial setup for testing some simple one sample t-test models
-----

### Description

* Set up a nipype workflow to use SPM12 to make second level models for _megameta_ task data (preprocessed using `batch8` SPM8 scripts) in BIDS derivative format   


### Setup

In [3]:
import os  # system functions

# NIYPE FUNCTIONS
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.algorithms.modelgen as model   # model specification
from nipype.interfaces.base import Bunch
from nipype.algorithms.misc import Gunzip

import scipy.io as sio
import numpy as np
import json
import pandas as pd

import random

from IPython.display import Image


from itertools import product

#### Matlab path


In [ ]:
# Set the way matlab should be called
mlab.MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")
# If SPM is not in your MATLAB path you should add it here
mlab.MatlabCommand.set_default_paths(PATH_TO_SPM_FOLDER)

In [ ]:
GROUP_DIR = '/data00/projects/megameta/group_models/'

#### Load JSON model config

In [ ]:
JSON_MODEL_FILE = os.path.join('/data00/projects/megameta/scripts/jupyter_megameta/second_level_models',
                               'model_specifications',
                               MODEL_SPEC_FILE)

In [ ]:
with open(JSON_MODEL_FILE) as fh:
    model_def = json.load(fh)

In [ ]:
MODEL_NAME = model_def['ModelName']

CONTRASTS = model_def['Contrasts']

ROOT_DIR = '/data00/projects/megameta'

In [ ]:
l2_contrast_list = CONTRASTS # list of specific contrast files to use in 2nd level model (include .nii?)


output_dir = os.path.join(GROUP_DIR,'derivatives', 'nipype','model_2nd-level_{}'.format(MODEL_NAME))        
working_dir = os.path.join(GROUP_DIR, 'working', 
                           'nipype', 'workingdir_model_2nd-level_{}'.format(MODEL_NAME))   

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
if not os.path.exists(working_dir):
    os.makedirs(working_dir)

## Get list of contrast files

In [1]:
def process_project(project_name, model_def=model_def, scan_all_subjs=False, DEBUG=False):
    
    project_spec = [pspec for pspec in model_def['Projects'] if pspec['Name']==project_name]
    
    if not project_spec:
        print('Cannot find specification for project: ', project_name)
        return None
    
    model_name = project_spec[0]['Model']
    cmap = project_spec[0]['ContrastMap']
    
    
    model_dir = os.path.join(ROOT_DIR, project_name, 
                             "derivatives", "nipype",
                             "model_{}".format(model_name)
                            )
    
    if not os.path.exists(model_dir):
        print('Cannot find first level model directory:', model_dir)
        return None
    
    subjs_with_models = [s for s in os.listdir(model_dir) if s.startswith('sub-')]
    
    if DEBUG:
        print("Found {} first level subject models\n".format(len(subjs_with_models)))
    
    
    contrast_lists = { cname: [] for cname in cmap}
    
    
    model_contrasts=None
    for sidx,subj in enumerate(subjs_with_models):
        
        if DEBUG:
            print('Processing',subj, '-',end='')
        
        first_level_dir = os.path.join(model_dir, subj, 'medium', 'fwhm_8')

        if scan_all_subjs or sidx==0:
            spm_mat_file = os.path.join(first_level_dir, 'SPM.mat')

            SPM = sio.loadmat(spm_mat_file, squeeze_me=True, struct_as_record=False)['SPM']

            model_contrasts = SPM.xCon

        if DEBUG:
            print(' found {} contrasts'.format(len(model_contrasts)))

        con_map = {con.name: 'con_{:0>4}.nii'.format(cidx) for cidx,con in enumerate(model_contrasts,1) }


        if DEBUG:
            print('\tContrasts are:', con_map)
        
        for model_con, proj_con in cmap.items():
            
            path_to_con = os.path.join(first_level_dir, con_map[proj_con])
            
            if os.path.exists(path_to_con):
                contrast_lists[model_con].append(path_to_con)
            
    return contrast_lists

## Define nodes

In [17]:
# Infosource - a function free node to iterate over the list of subject names
l2_infosource = pe.Node(util.IdentityInterface(fields=['contrast_id']),
                  name="infosource")

smoothing_kernels = [ 8 ]
resolutions = ['medium']

resolution_and_kernel_list = product(resolutions, smoothing_kernels)


l2_infosource.iterables = [('contrast_id', l2_contrast_list), 
                           ('resolution_and_smoothing', resolution_and_kernel_list)
                        ]

NameError: name 'l2_contrast_list' is not defined

In [ ]:
# SelectFiles - to grab the data (alternativ to DataGrabber)

subject_pattern='*'
OUTPUT_DIR = output_dir
l2_output_dir = output_dir

l2_templates = {'cons': os.path.join(output_dir, MODEL_NAME, subject_pattern, '{smoothing_ksize}',
                         '{contrast_id}.nii')}

l2_selectfiles = pe.Node(nio.SelectFiles(l2_templates,
                               base_directory=OUTPUT_DIR,
                               sort_filelist=True),
                   name="selectfiles")

In [4]:
def make_contrast_list(model_path, cname='puremessageXselfrel', sample=10):

    import json
    import random
    import os
    import scipy.io as sio
    
    ROOT_DIR = '/data00/projects/megameta'
    
    def process_project(project_name, model_def, scan_all_subjs=False, DEBUG=False):

        project_spec = [pspec for pspec in model_def['Projects'] if pspec['Name']==project_name]

        if not project_spec:
            print('Cannot find specification for project: ', project_name)
            return None

        model_name = project_spec[0]['Model']
        cmap = project_spec[0]['ContrastMap']


        model_dir = os.path.join(ROOT_DIR, project_name, 
                                 "derivatives", "nipype",
                                 "model_{}".format(model_name)
                                )

        if not os.path.exists(model_dir):
            print('Cannot find first level model directory:', model_dir)
            return None

        subjs_with_models = [s for s in os.listdir(model_dir) if s.startswith('sub-')]

        if DEBUG:
            print("Found {} first level subject models\n".format(len(subjs_with_models)))


        contrast_lists = { cname: [] for cname in cmap}


        model_contrasts=None
        for sidx,subj in enumerate(subjs_with_models):

            if DEBUG:
                print('Processing',subj, '-',end='')

            first_level_dir = os.path.join(model_dir, subj, 'medium', 'fwhm_8')

            if scan_all_subjs or sidx==0:
                spm_mat_file = os.path.join(first_level_dir, 'SPM.mat')

                SPM = sio.loadmat(spm_mat_file, squeeze_me=True, struct_as_record=False)['SPM']

                model_contrasts = SPM.xCon

            if DEBUG:
                print(' found {} contrasts'.format(len(model_contrasts)))

            con_map = {con.name: 'con_{:0>4}.nii'.format(cidx) for cidx,con in enumerate(model_contrasts,1) }


            if DEBUG:
                print('\tContrasts are:', con_map)

            for model_con, proj_con in cmap.items():

                path_to_con = os.path.join(first_level_dir, con_map[proj_con])

                if os.path.exists(path_to_con):
                    contrast_lists[model_con].append(path_to_con)

        return contrast_lists
    
    
    
    
    with open(model_path) as fh:
        model_def = json.load(fh)
        
    conlist=[]
    for p in model_def['Projects']:
        conlist.extend(random.sample(process_project(p['Name'], model_def)[cname], sample))
    
    return conlist, {}

In [ ]:
l2_getcontrasts = pe.Node(util.Function(input_names=['model_path'],
                                     output_names=['contrasts', 'covariates'],
                                      function=make_contrast_list),
                    name='makecontrasts')

In [ ]:
# Datasink - creates output folder for important outputs
datasink = pe.Node(nio.DataSink(base_directory=OUTPUT_DIR,
                         container=l2_output_dir),
                name="datasink")

## Model nodes

In [ ]:
osttdesign = pe.Node(spm.model.OneSampleTTestDesign(),
                         name="osttdesign")

osttdesign.inputs.explicit_mask_file='/data00/tools/spm8/apriori/brainmask_th25.nii'

In [1]:
# Multiple Regression Design - creates mreg Design
mregdesign = pe.Node(spm.model.MultipleRegressionDesign(),
                         name="mregdesign")

#mregdesign.inputs.covariates=covs

'''
covs = [
    {'name':'QuitIntent','vector': [2,2,3,2,3]},
    {'name':'FTND','vector': [6,3,4,2,3]},
    {'name':'mean_WC','vector': [77.92307692,  30.34782609,  51.2173913, 41.30434783,  41.0 ]},
]
'''

mregdesign.inputs.explicit_mask_file='/data00/tools/spm8/apriori/brainmask_th25.nii'

NameError: name 'pe' is not defined

In [ ]:
# EstimateModel - estimate the parameters of the model
level2estimate = pe.Node(spm.model.EstimateModel(estimation_method={'Classical': 1}),
                      name="level2estimate")



In [ ]:
# EstimateContrast - estimates simple group contrast
level2conestimate = pe.Node(spm.model.EstimateContrast(group_contrast=True),
                         name="level2conestimate")


In [ ]:
# NEED FUNCTION TO CREATE CONTRASTS
# should do this by reading the first level SPM.mat file
# and generating contrast codes

# for MREG models
# need to extend syntax of JSON config

In [ ]:
'''
cont1 = ['QuitIntent', 'T', ['QuitIntent', 'FTND', 'mean_WC', 'mean'], [1, 0, 0, 0]]
cont2 = ['FTND', 'T', ['QuitIntent', 'FTND', 'mean_WC', 'mean'], [0, 1, 0, 0]]
cont3 = ['mean_WC', 'T', ['QuitIntent', 'FTND', 'mean_WC', 'mean'], [0, 0, 1, 0]]
cont4 = ['mean', 'T', ['QuitIntent', 'FTND', 'mean_WC', 'mean'], [0, 0, 0, 1]]
'''

cont = ['Group', 'T', ['mean'], [1]]

level2conestimate.inputs.contrasts = [cont]

## Setup second level workflow

In [ ]:
#l2_working_dir = os.path.join(PROJECT_DIR, 'nipype', 'workingdir_banner_2nd_level')
l2_working_dir = working_dir

In [ ]:

l2analysis = pe.Workflow(name='l2analysis')

l2analysis.base_dir = l2_working_dir

# Connect up the 2nd-level analysis components
l2analysis.connect(
                    [
                        
                       # (l2_infosource, l2_getcontrasts, [('contrast_id', 'contrast_id'),
                        #                               ('model_path')]),
                     
                     (l2_getcontrasts,  mregdesign, [('contrasts', 'in_files')]),
                     
                    (mregdesign, level2estimate, [('spm_mat_file',
                                                          'spm_mat_file')] ),
                    (level2estimate, level2conestimate, [('spm_mat_file',
                                                          'spm_mat_file'),
                                                         ('beta_images',
                                                          'beta_images'),
                                                         ('residual_image',
                                                          'residual_image')])
                    ])
